In [2]:
%matplotlib inline
import numpy as np
from gPhoton.pipeline import execute_pipeline
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
from gPhoton.io.fits_utils import get_fits_data, read_wcs_from_fits
from astropy.visualization import ZScaleInterval
import os
import tqdm
import imageio.v2 as imageio
from IPython.display import Image
import astropy.wcs
from gPhoton.coadd import make_shared_wcs, project_to_shared_wcs, bin_projected_weights

Pipeline steps:
1. Run photometry with extraction on NUV
2. Run photometry with extraction on FUV
3. Run photometry on FUV with NUV source positions
4. Run photometry on NUV with FUV source positions
5. Generate a merged catalog from NUV extractions
6. Generate a merged catalog from FUV extractions
7. Generate a merged catalog from all NUV / FUV detections

In [3]:
from glcat_merge_utils import *
from lightcurve_utils import *

In [51]:
eclipse = 23456
datadir = "/Users/cm/github/gphoton2_refactor/gPhoton2/test_data/"
edir = f"e{str(eclipse).zfill(5)}"
apers = [1.5, 2.3, 3.8, 6.0, 9.0, 12.8, 17.3]
at = {}
for p in 'nf':
    for m in 'nf':
        at[f'{p}{m}'] = {}
        for a in apers:
            at[f'{p}{m}'][a] = pd.read_csv(
                f"{datadir}{edir}/{edir}-{p}d-b00-f0120-movie-photom-{str(a).replace('.','_')}-mo{m}.csv",
                index_col=None)


#ff = pd.read_csv(f"{datadir}{edir}/{edir}-fd-b00-f0120-movie-photom-12_8-mof.csv",index_col=None)
#nf = pd.read_csv(f"{datadir}{edir}/{edir}-nd-b00-f0120-movie-photom-12_8-mof.csv",index_col=None)
#fn = pd.read_csv(f"{datadir}{edir}/{edir}-fd-b00-f0120-movie-photom-12_8-mon.csv",index_col=None)

expts = {'NUV':pd.read_csv(f"{datadir}{edir}/{edir}-nd-b00-f0120-movie-exptime.csv",index_col=None),
         'FUV':pd.read_csv(f"{datadir}{edir}/{edir}-fd-b00-f0120-movie-exptime.csv",index_col=None)}

In [19]:
primary_catalog_columns = [
    #'ra','dec',
    #'xcentroid','ycentroid',
    'xcenter','ycenter',
    #'elongation','eccentricity',
    #'max_value','maxval_xindex','maxval_yindex',
    #'min_value','minval_xindex','minval_yindex',
    'extended_source',
    #'aperture_sum',
    #'aperture_sum_mask','aperture_sum_edge',
]

matched_catalog_columns = [
    #'ra','dec',
    #'xcentroid','ycentroid',
    'xcenter','ycenter',
    #'aperture_sum',
    #'aperture_sum_mask','aperture_sum_edge',
]

In [73]:
def derived_photometry_table(catalog_table,exposure_table,band):
    assert band in ['NUV','FUV']
    expt = exposure_table.expt.sum()
    cps = np.array(catalog_table.aperture_sum.tolist())/expt
    cps_err = np.sqrt(catalog_table.aperture_sum.tolist())/expt
    mag = counts2mag(cps,band)
    mag_err = np.abs(counts2mag(cps-cps_err,band) - mag)
    return pd.DataFrame(
        {'CPS':cps,'CPS_ERR':cps_err,
         'MAG':mag,'MAG_ERR':mag_err,
         #'EXPT':expt,
         'MASK_FLAG':(catalog_table['aperture_sum_mask'] != 0).astype(int),
         'EDGE_FLAG':(catalog_table['aperture_sum_edge'] != 0).astype(int),}
    )
    aperture_sum_mask

In [76]:
band = 'NUV'
expts[band]['expt'].sum()

1500.2286768752826

In [106]:
def accumulate_photometry(at,c,expts):
    band = {'n':'NUV','f':'FUV'}[c[0]]
    phot = pd.DataFrame()
    for i,a in enumerate(apers):
        suffix = f'_APER{i}'
        phot[f'{band}_SUM{suffix}']=list(at[f'{c}'][a]['aperture_sum'])
        phot[f'{band}_EDGE{suffix}']=list((at[f'{c}'][a]['aperture_sum_edge'] != 0).astype(int))
        phot[f'{band}_MASK{suffix}']=list((at[f'{c}'][a]['aperture_sum_mask'] != 0).astype(int))
        dp = derived_photometry_table(at[f'{c}'][a],expts[band],band)
        phot = pd.concat([phot,dp.add_prefix(f'{band}_').add_suffix(suffix)],axis=1)
    phot[f'{band}_EXPT']=np.full(len(at[f'{c}'][a]),expts[band]['expt'].sum())
    return pd.DataFrame(phot)
aperture_sums = accumulate_photometry(at,'nn',expts)
aperture_sums

,NUV_SUM_APER0,NUV_EDGE_APER0,NUV_MASK_APER0,NUV_CPS_APER0,NUV_CPS_ERR_APER0,NUV_MAG_APER0,NUV_MAG_ERR_APER0,NUV_MASK_FLAG_APER0,NUV_EDGE_FLAG_APER0,NUV_SUM_APER1,...,NUV_SUM_APER6,NUV_EDGE_APER6,NUV_MASK_APER6,NUV_CPS_APER6,NUV_CPS_ERR_APER6,NUV_MAG_APER6,NUV_MAG_ERR_APER6,NUV_MASK_FLAG_APER6,NUV_EDGE_FLAG_APER6,NUV_EXPT
0,74.195186,1,0,0.049456,0.005742,23.344454,0.133986,0,1,158.509155,...,3112.342393,1,0,2.074579,0.037187,19.287675,0.019638,0,1,1500.228677
1,35.807291,0,0,0.023868,0.003989,24.135465,0.198537,0,0,75.978951,...,2269.889814,1,0,1.513029,0.031757,19.630382,0.023031,0,1,1500.228677
2,114.462236,0,0,0.076297,0.007131,22.873738,0.106544,0,0,221.705575,...,2983.111298,1,0,1.988438,0.036406,19.333720,0.020063,0,1,1500.228677
3,50.258974,0,0,0.033501,0.004726,23.767360,0.165089,0,0,116.549745,...,2375.268322,0,0,1.583271,0.032486,19.581112,0.022509,0,0,1500.228677
4,33.605723,0,0,0.022400,0.003864,24.204361,0.205582,0,0,80.068402,...,2142.556936,0,0,1.428154,0.030854,19.693063,0.023713,0,0,1500.228677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,63.903954,0,0,0.042596,0.005329,23.506574,0.145096,0,0,147.978526,...,11867.990288,0,1,7.910788,0.072616,17.834451,0.010012,1,0,1500.228677
4836,80.647389,0,1,0.053757,0.005986,23.253918,0.128178,1,0,201.440690,...,29112.805548,0,1,19.405579,0.113732,16.860184,0.006382,1,0,1500.228677
4837,47.527211,0,0,0.031680,0.004595,23.828038,0.170153,0,0,115.656381,...,6974.056362,0,1,4.648662,0.055665,18.411680,0.013080,1,0,1500.228677
4838,51.393869,0,1,0.034257,0.004779,23.743115,0.163111,1,0,127.719978,...,8261.524717,1,1,5.506844,0.060586,18.227743,0.012011,1,1,1500.228677


In [118]:
def make_catalog_by_code(at,c,expts,eclipse):
    band = {'n':'NUV','f':'FUV'}[c[0]]
    cat = pd.concat([pd.DataFrame({'eclipse':np.full(len(at[f'{c}'][12.8]),eclipse),}),
                     at[f'{c}'][12.8][['ra','dec']],
                     at[f'{c}'][12.8][['xcenter','ycenter']].add_prefix(f'{band}_'),
                     accumulate_photometry(at,c,expts),],axis=1)
    if len(set(c))==1: # only the primary / extracted catalog contains extended source information
        cat = pd.concat([cat,pd.DataFrame({f'{band}_EXTENDED':
                                           list(at[f'{c}'][12.8]['extended_source'])})],axis=1)
    return cat

def make_catalog(at,b,expts,eclipse):
    assert b in ['n','f']
    c = f'{b}{b}'
    cat_1 = make_catalog_by_code(at,f'{c}',expts,eclipse)
    c = ({"f":"n","n":"f"}[b])+b # there must be a more elegant way to do this
    cat_2 = make_catalog_by_code(at,f'{c}',expts,eclipse)
    return pd.concat([cat_1,cat_2],axis=1).rename(columns=str.upper)

catalog = make_catalog(at,'n',expts,eclipse)
catalog

,ECLIPSE,RA,DEC,NUV_XCENTER,NUV_YCENTER,NUV_SUM_APER0,NUV_EDGE_APER0,NUV_MASK_APER0,NUV_CPS_APER0,NUV_CPS_ERR_APER0,...,FUV_SUM_APER6,FUV_EDGE_APER6,FUV_MASK_APER6,FUV_CPS_APER6,FUV_CPS_ERR_APER6,FUV_MAG_APER6,FUV_MAG_ERR_APER6,FUV_MASK_FLAG_APER6,FUV_EDGE_FLAG_APER6,FUV_EXPT
0,23456,323.448811,-2.638534,1638.452026,170.996174,74.195186,1,0,0.049456,0.005742,...,209.245330,1,0,0.124685,0.008620,21.080466,0.077778,0,1,1678.19451
1,23456,323.458572,-2.636944,1615.050994,174.814059,35.807291,0,0,0.023868,0.003989,...,295.241799,1,0,0.175928,0.010239,20.706661,0.065101,0,1,1678.19451
2,23456,323.448619,-2.635363,1638.913971,178.608991,114.462236,0,0,0.076297,0.007131,...,186.479726,1,0,0.111119,0.008137,21.205527,0.082569,0,1,1678.19451
3,23456,323.509375,-2.632623,1493.247071,185.185578,50.258974,0,0,0.033501,0.004726,...,351.702132,0,0,0.209572,0.011175,20.516668,0.059495,0,0,1678.19451
4,23456,323.459455,-2.632761,1612.932867,184.853847,33.605723,0,0,0.022400,0.003864,...,266.103516,0,0,0.158565,0.009720,20.819479,0.068685,0,0,1678.19451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,23456,323.546199,-1.505455,1404.865650,2890.475707,63.903954,0,0,0.042596,0.005329,...,276.932677,0,0,0.165018,0.009916,20.776170,0.067286,0,0,1678.19451
4836,23456,323.548553,-1.504981,1399.217901,2891.612115,80.647389,0,1,0.053757,0.005986,...,293.286122,0,0,0.174763,0.010205,20.713877,0.065325,0,0,1678.19451
4837,23456,323.543522,-1.505130,1411.288694,2891.256176,47.527211,0,0,0.031680,0.004595,...,291.637053,0,0,0.173780,0.010176,20.719999,0.065515,0,0,1678.19451
4838,23456,323.541941,-1.501864,1415.081510,2899.095427,51.393869,0,1,0.034257,0.004779,...,302.638096,0,0,0.180336,0.010366,20.679797,0.064277,0,0,1678.19451


In [97]:
set('nn')==1

False

In [113]:
b = 'n'
c = ({"f":"n","n":"f"}[b])+b
c

'fn'